In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install mtcnn
import tensorflow as tf
import keras
from keras.preprocessing import image as kimage

from mtcnn import MTCNN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import time
import os

     |████████████████████████████████| 32.7 MB 160 kB/s 
     |████████████████████████████████| 2.3 MB 5.3 MB/s 


In [1]:
# get current working directory
CWD = os.getcwd()

# model directory
MODEL = CWD + "\\thesis_final_model.h5"

# template folder for flask
FLASK_FOLDER = CWD + "\\template"

C:\Users\Mark\Downloads\drive-download-20220523T092121Z-001\Thesis Prototype App


In [8]:
print(os.listdir(CWD))
print(CWD)

['.ipynb_checkpoints', 'App Demo', 'template', 'Thesis Flask App.ipynb', 'thesis_final_model.h5']
C:\Users\Mark\Downloads\drive-download-20220523T092121Z-001\Thesis Prototype App


In [ ]:
class Emotion_Analysis():
    def __init__(self):
        self.model = keras.models.load_model(MODEL)
        self.objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    
    def detect_emotion(self, img):
        img = Image.fromarray(img)
        img = img.resize((48,48))
        img = kimage.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img /= 255.0
        # v = []
        prediction = self.model.predict(img)
        dominant, sub_dominant = getIndex(prediction[0])
        # idx = np.argmax(prediction[0])
        # for i in prediction[0]:
        #   v.append(i)
        # print(v)
        return self.objects[dominant], self.objects[sub_dominant]

In [ ]:
def getIndex(lst):
  tmp = list(lst)
  lst.sort()
  return tmp.index(lst[-1]), tmp.index(lst[-2])


In [ ]:
data = {
      'hope': [],
      'pride': [],
      'enjoyment': [],
      'angry': [],
      'anxiety': [],
      'shame': [],
      'boredom': [],
      'hopelessness': [],
      'disgust': [],
      'fear': [],
      'sad': [],
      'surprise': [],
      'neutral': [],
      'people': []
  }
total_num_of_faces = []

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://nddabcgkyi-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:

def getAffectiveState(dominant, sub_dominant):
  state = ''
  if dominant == 'happy' and sub_dominant == 'surprise': 
    state = 'hope'
  elif dominant == 'happy' and sub_dominant == 'angry': 
    state = 'pride'
  elif dominant == 'happy': 
    state = 'enjoyment'
  elif dominant == 'angry': 
    state = 'angry'
  elif dominant == 'fear' and sub_dominant == 'surprise': 
    state = 'anxiety'
  elif dominant == 'fear' and sub_dominant == 'sad': 
    state = 'shame'
  elif dominant == 'sad' and sub_dominant == 'neutral' or sub_dominant == 'surprise': 
    state = 'boredom'
  elif dominant == 'sad' and sub_dominant == 'surprise': 
    state = 'hopelessness'
  else:
    state = dominant
  return state


In [ ]:
def getDict():
  return {
      'hope': 0,
      'pride': 0,
      'enjoyment': 0,
      'angry': 0,
      'anxiety': 0,
      'shame': 0,
      'boredom': 0,
      'hopelessness': 0,
      'disgust': 0,
      'fear': 0,
      'sad': 0,
      'surprise': 0,
      'neutral': 0,
      'people': 0
  }

In [ ]:
from flask import Flask, render_template, request, make_response, jsonify
import base64
import cv2
import numpy as np
import datetime
import random

app = Flask(__name__, template_folder=FLASK_FOLDER)
face_detector = MTCNN()
emotion_detector = Emotion_Analysis()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':

        fs = request.files.get('snap')

        if fs:
            start_time = time.time()

            img = cv2.imdecode(np.frombuffer(fs.read(), np.uint8), cv2.IMREAD_UNCHANGED)
            imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            faces = face_detector.detect_faces(imgRGB)
            affective_state = getDict()

            for face in faces:
                x,y,w,h = face['box'][0], face['box'][1], face['box'][2], face['box'][3]
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

                cropped_image_gray = imgGray[y:y+h, x:x+w]

                dominant, sub_dominant = emotion_detector.detect_emotion(cropped_image_gray)
                state = getAffectiveState(dominant, sub_dominant)
                affective_state[state] += 1

                cv2.putText(img, state, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36,255,12), 2) 
            
            ret, buf = cv2.imencode('.jpg', img)
            image_url = base64.b64encode(buf).decode('ascii')
            
            num_of_faces = len(faces)
            total_num_of_faces.append(num_of_faces)
            affective_state['people'] = num_of_faces
            
            for k,v in affective_state.items():
              data[k].append(v)
            print(affective_state)
            # cv2_imshow(img)
            print("--- %s seconds ---" % (time.time() - start_time))
            
            return jsonify({
                'image_url': image_url,
                'affective_state': affective_state
            })
        else:
            return 'You forgot Snap!'

    return 'Hello World!'

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2022 02:28:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2022 02:28:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Apr/2022 02:28:34] "POST /upload HTTP/1.1" 200 -


{'hope': 0, 'pride': 0, 'enjoyment': 2, 'angry': 3, 'anxiety': 0, 'shame': 0, 'boredom': 2, 'hopelessness': 0, 'disgust': 0, 'fear': 4, 'sad': 1, 'surprise': 0, 'neutral': 3, 'people': 15}
--- 2.7507195472717285 seconds ---


127.0.0.1 - - [10/Apr/2022 02:28:40] "POST /upload HTTP/1.1" 200 -


{'hope': 0, 'pride': 0, 'enjoyment': 1, 'angry': 2, 'anxiety': 0, 'shame': 0, 'boredom': 3, 'hopelessness': 0, 'disgust': 0, 'fear': 2, 'sad': 2, 'surprise': 1, 'neutral': 6, 'people': 17}
--- 2.43074107170105 seconds ---


...

127.0.0.1 - - [10/Apr/2022 02:31:36] "POST /upload HTTP/1.1" 200 -


{'hope': 0, 'pride': 0, 'enjoyment': 0, 'angry': 1, 'anxiety': 0, 'shame': 0, 'boredom': 2, 'hopelessness': 0, 'disgust': 0, 'fear': 1, 'sad': 5, 'surprise': 0, 'neutral': 5, 'people': 14}
--- 1.993779182434082 seconds ---


127.0.0.1 - - [10/Apr/2022 02:31:39] "POST /upload HTTP/1.1" 200 -


{'hope': 0, 'pride': 0, 'enjoyment': 1, 'angry': 0, 'anxiety': 0, 'shame': 0, 'boredom': 0, 'hopelessness': 0, 'disgust': 0, 'fear': 3, 'sad': 5, 'surprise': 0, 'neutral': 5, 'people': 14}
--- 1.9569940567016602 seconds ---


127.0.0.1 - - [10/Apr/2022 02:31:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2022 02:31:42] "GET /favicon.ico HTTP/1.1" 404 -
